<a href="https://colab.research.google.com/github/vixbi/actprobl/blob/main/Shumakova_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install razdel

In [ ]:
import pandas as pd
import razdel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

## **Задание 1**

In [ ]:
# векторизация

data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True,
                               random_state = 500)

# дефолтный векторайзер из sklearn
vectorizer_default = CountVectorizer()

# кастомный токенизатор для векторайзера
def r_t(text):
    return [word.text.lower() for word in list(razdel.tokenize(text))]

vectorizer_razdel = CountVectorizer(tokenizer=r_t)

# векторизация по дефолту
X_train_default = vectorizer_default.fit_transform(train.comment)
X_test = vectorizer_default.transform(test.comment)

# # векторизация с кастомным токенизатором
X_train_razdel = vectorizer_razdel.fit_transform(train.comment)
X_test_razdel = vectorizer_razdel.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# логистическая регрессия: дефолтный векторайзер
%%time

class_default = LogisticRegression()
class_default.fit(X_train_default, y)
pred_default = class_default.predict(X_test)
print(classification_report(y_test, pred_default, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90       983
         1.0       0.83      0.67      0.74       459

    accuracy                           0.85      1442
   macro avg       0.85      0.80      0.82      1442
weighted avg       0.85      0.85      0.85      1442

CPU times: user 2.65 s, sys: 1.02 s, total: 3.67 s
Wall time: 1.92 s


In [ ]:
# логистическая регрессия: кастомный векторайзер
%%time

class_razdel = LogisticRegression()
class_razdel.fit(X_train_razdel, y)
pred_razdel = class_razdel.predict(X_test_razdel)
print(classification_report(y_test, pred_razdel, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90       983
         1.0       0.84      0.69      0.76       459

    accuracy                           0.86      1442
   macro avg       0.85      0.81      0.83      1442
weighted avg       0.86      0.86      0.86      1442

CPU times: user 4.09 s, sys: 1.41 s, total: 5.5 s
Wall time: 2.85 s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Не очень понятно, как тут выбирать победителя, потому что тут буквально отличие в 0.01... Ну я посчитаю, что выиграла дефолтная токенизация :) Тем более, она выполняется быстрее раза в 3-4. Ну и раз уж мы заговорили о времени выполнения...

In [ ]:
# штука из интереса; прошу прощения за нецензурную брань в коде, эксперимент был
# сделан с научной целью
%%time

l = [i for i in range(14412)]
l_train, l_test = train_test_split(l, test_size=0.1, shuffle=True,
                               random_state = 512)
preds_int = []
for i in l_test:
  if 'хохол' in data.comment[i] or 'хохл' in data.comment[i] \
   or 'дебил' in data.comment[i] or 'хуй' in data.comment[i] \
    or 'бля' in data.comment[i] or  'ебнут' in data.comment[i] \
    or 'ебанут' in data.comment[i] or  'пизд' in data.comment[i] \
    or  'конч' in data.comment[i] or  'мудак' in data.comment[i]:
    preds_int.append(float(1))
  else:
    preds_int.append(float(0))

print(classification_report(y_test, preds_int, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.69      0.90      0.78       983
         1.0       0.37      0.12      0.19       459

    accuracy                           0.65      1442
   macro avg       0.53      0.51      0.48      1442
weighted avg       0.59      0.65      0.59      1442

CPU times: user 143 ms, sys: 207 µs, total: 143 ms
Wall time: 143 ms


Результаты, конечно, не такие outstanding, как у предыдущих двух моделей, но в принципе терпимые (70% угадано правильно!), а еще оно сильно быстрее.

# **Задание 2**

In [ ]:
# стоп-слова для русского языка
!wget -O stop.txt https://raw.githubusercontent.com/negapedia/nltk/master/corpora/stopwords/russian
with open('stop.txt', encoding='utf-8') as file:
  s_w = [line.strip() for line in file]

--2024-11-24 10:45:21--  https://raw.githubusercontent.com/negapedia/nltk/master/corpora/stopwords/russian
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4544 (4.4K) [text/plain]
Saving to: ‘stop.txt’

stop.txt            100%[===================>]   4.44K  --.-KB/s    in 0s      

2024-11-24 10:45:21 (35.0 MB/s) - ‘stop.txt’ saved [4544/4544]



In [ ]:
# обучающую и тестовую выборки я оставлю как в предыдущих пунктах
# логистическая регрессия + CountVectorizer
%%time

# я буду использовать одни и те же параметры для обоих векторайзеров, чтобы
# потом было попроще сравнивать

vectorizer_lg = CountVectorizer(stop_words=s_w, binary=True, min_df=0.0001,
                                ngram_range=(1,7), strip_accents="unicode")
X_train_lg = vectorizer_lg.fit_transform(train.comment)
X_test_lg = vectorizer_lg.transform(test.comment)

class_lg_cv = LogisticRegression(C=0.1, class_weight="balanced")
class_lg_cv.fit(X_train_lg, y)
pred_lg_cv = class_lg_cv.predict(X_test_lg)
print(classification_report(y_test, pred_lg_cv, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['важныи', 'восемнадцатыи', 'восьмои', 'второи', 'двадцатыи', 'двенадцатыи', 'девятнадцатыи', 'девятыи', 'деиствительно', 'десятыи', 'другои', 'еи', 'каждыи', 'какои', 'которои', 'которыи', 'многочисленныи', 'мнои', 'мое', 'неи', 'одиннадцатыи', 'однои', 'первыи', 'пожалуиста', 'пятнадцатыи', 'пятыи', 'самои', 'своеи', 'сеаои', 'седьмои', 'сеичас', 'семнадцатыи', 'собои', 'такои', 'твое', 'твои', 'тобои', 'третии', 'тринадцатыи', 'четвертыи', 'четырнадцатыи', 'шестнадцатыи', 'шестои', 'этои'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.92      0.81      0.86       983
         1.0       0.67      0.84      0.75       459

    accuracy                           0.82      1442
   macro avg       0.79      0.82      0.80      1442
weighted avg       0.84      0.82      0.82      1442

CPU times: user 4.97 s, sys: 693 ms, total: 5.66 s
Wall time: 5.18 s


In [ ]:
# KNN + TF-IDF

%%time

vectorizer_dtc = TfidfVectorizer(stop_words=s_w, binary=True, min_df=0.0001,
                                 strip_accents="unicode")
X_train_dtc = vectorizer_dtc.fit_transform(train.comment)
X_test_dtc = vectorizer_dtc.transform(test.comment)

class_dtc_tfidf = KNeighborsClassifier(weights='distance',
                                       n_neighbors=10, metric='cosine')
class_dtc_tfidf.fit(X_train_dtc, y)
pred_dtc_tfidf = class_dtc_tfidf.predict(X_test_dtc)
print(classification_report(y_test, pred_dtc_tfidf, zero_division=0))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['важныи', 'восемнадцатыи', 'восьмои', 'второи', 'двадцатыи', 'двенадцатыи', 'девятнадцатыи', 'девятыи', 'деиствительно', 'десятыи', 'другои', 'еи', 'каждыи', 'какои', 'которои', 'которыи', 'многочисленныи', 'мнои', 'мое', 'неи', 'одиннадцатыи', 'однои', 'первыи', 'пожалуиста', 'пятнадцатыи', 'пятыи', 'самои', 'своеи', 'сеаои', 'седьмои', 'сеичас', 'семнадцатыи', 'собои', 'такои', 'твое', 'твои', 'тобои', 'третии', 'тринадцатыи', 'четвертыи', 'четырнадцатыи', 'шестнадцатыи', 'шестои', 'этои'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.88      0.90      0.89       983
         1.0       0.77      0.73      0.75       459

    accuracy                           0.84      1442
   macro avg       0.82      0.81      0.82      1442
weighted avg       0.84      0.84      0.84      1442

CPU times: user 1.29 s, sys: 182 ms, total: 1.48 s
Wall time: 1.48 s


In [ ]:
# LogReg + CV top 10 toxic comments

a = class_lg_cv.predict_proba(X_test_lg)

top_most_toxic = list([(test.iloc[i]['comment'].strip(),a[i][1]) for i in range(len(a))])
sorted(top_most_toxic, key=lambda x: x[1], reverse=True)[:10]

[('самый сброд червей-пидоров Не, ну если это черви-пидоры, то перечисленные тобой далее люди - это черви-пидоры в гуголплексе стаса какай просто Автор абсолютно неинтересного технического блога на Ютабе, пускай честного в отличие от Виласкома, но от того ничуть не менее унылого, любитель шутечек про куканы и поопускать маргиналов на тему не присаживал на них тёлку . соболева Говорящая голова народных масс, своего мнения не имеет, никогда в жизни не сделает ролик по какому-либо малооглашённому поводу, по хайповому не выскажет какую-либо точку зрения кроме той, которая будет угодно большинству. В жизни, по описанию очевидцев, быдловат. Продавал жопу Собянину, чем, ЧСХ, не может похвастаться никто из ОП-треда, кроме Хованского. амирана из дневника хача Никогда не понимал, кому он вообще нахуй нужен. Кому на полном серьёзе может доставлять наблюдать за жизнью мажора-рабовладельца, не сообщающего вам ничего полезного, или хотя бы как жить так же? Летсплеи на сто порядков более высокоинтелл

In [ ]:
# KNN + TF-IDF top 10 toxic comments

a = class_dtc_tfidf.predict_proba(X_test_dtc)

top_most_toxic = list([(test.iloc[i]['comment'].strip(),a[i][1]) for i in range(len(a))])
sorted(top_most_toxic, key=lambda x: x[1], reverse=True)[:10]

[('мандан против хохлов?', 1.0),
 ('Какие же ватники наивные и тупые.', 1.0),
 ('соси как хохлы сосут хохол', 1.0),
 ('Хорс пидор, тарков не дал!', 1.0),
 ('Они поддерживали свой народ, они же националисты, ты забыл, даун? Путина они не поддерживали.',
  1.0),
 ('Тупа сеги трэнду бездарного хуйсоса, а так жи насрал на ебала его зрителям, десу.',
  1.0),
 ('долбоебы, лезущие в опасные места ебаные ублюдки альпинисты создают своим неадекватным поведением по итогу проблемы не только своим родным и близким, но и вообще всем вокруг больным людям нехуй делать Ох уж эти визги тепличных корзинок. Ваши предки, полезшие в заснеженные рашкинские болота и выжившие среди этой грязи, медведей и комаров с единственным изобретением - факелом, харкают вам на ебало, никчемные вырожденцы.',
  1.0),
 ('Хохлы то да разъезжаются ты прав.', 1.0),
 ('В РОТ ИБАЛ ТАКИЕ ПОЗДРАВЛЕНИЯ', 1.0),
 ('ГОВНО СОЖРУ, ЛИШЬ БЫ ЛИБЕРАХИ ГОРЕЛИ НА БУТЫЛКУ СЯДУ, ЛИШЬ БЫ ЛИБЕРАХИ ГОРЕЛИ СДОХНУ, ЛИШЬ БЫ ЛИБЕРАХИ ГОРЕЛИ',
  1.0)]

Пу-пу-пу. Ну, что хочется сказать.


1.   Все комментарии разные, пересечений нет
2.   Основное отличие - длина комментариев. Логрег явно выдал более длинные штуки, а KNN - более короткие. Наверное, это можно объяснить особенностями самих моделей.
3.  Логрег оказался менее радикальным, и даже для самых токсичных комментариев у него нет значений вероятности, равных единице. А у KNN есть.
4.  Тексты правда токсичные. :)



# **Задание 3**
Сразу скажу, что f1 для каждого классификатора не получилась прям 0.75, я оставила близкие значения.

In [ ]:
# теперь для всего будет единый векторайзер на основе CV; стоп-слова удалены
vectorizer = CountVectorizer(stop_words=s_w, binary=True, min_df=0.0001,
                                ngram_range=(1,7), strip_accents="unicode")
X_train = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

### **Логистическая регрессия**

In [ ]:
# Logistic Regression

class_lg = LogisticRegression(C=0.1, class_weight="balanced")
class_lg.fit(X_train, y)
pred_lg = class_lg.predict(X_test)
print(classification_report(y_test, pred_lg, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.81      0.86       983
         1.0       0.67      0.84      0.75       459

    accuracy                           0.82      1442
   macro avg       0.79      0.82      0.80      1442
weighted avg       0.84      0.82      0.82      1442



In [ ]:
voc = list(vectorizer.get_feature_names_out())
words_importance_lg = list([(voc[i], abs(class_lg.coef_[0][i]))
                        for i in range(len(class_lg.coef_[0]))])
sorted(words_importance_lg, key=lambda x: x[1], reverse=True)[:5]

[('хохлы', 1.5667001719638023),
 ('хохлов', 1.4930805639943185),
 ('нахуи', 1.1872176315151637),
 ('блядь', 1.1017294333678695),
 ('дебил', 1.0523581630800511)]

### **Дерево решений**

In [ ]:
# Decision Tree

class_dt = DecisionTreeClassifier(max_depth=1000, criterion="log_loss",
                                  class_weight='balanced')
class_dt.fit(X_train, y)
pred_dt = class_dt.predict(X_test)
print(classification_report(y_test, pred_dt, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.71      0.78       983
         1.0       0.55      0.77      0.64       459

    accuracy                           0.73      1442
   macro avg       0.71      0.74      0.71      1442
weighted avg       0.77      0.73      0.74      1442



In [ ]:
len(class_dt.feature_importances_)

46106

In [ ]:
words_importance_dt = list([(voc[i], abs(class_dt.feature_importances_[i]))
                        for i in range(len(class_dt.feature_importances_))])
sorted(words_importance_dt, key=lambda x: x[1], reverse=True)[:5]

[('хохлы', 0.013862429407573297),
 ('хохлов', 0.011199951939501145),
 ('нахуи', 0.010603811786239623),
 ('например', 0.00800354988797298),
 ('сука', 0.007224198761708376)]

### **Наивный Байес**

In [ ]:
# Naive Bayes
class_nb = MultinomialNB(alpha=1.)
class_nb.fit(X_train, y)
pred_nb = class_nb.predict(X_test)
print(classification_report(y_test, pred_nb, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.97      0.91       983
         1.0       0.90      0.63      0.75       459

    accuracy                           0.86      1442
   macro avg       0.88      0.80      0.83      1442
weighted avg       0.87      0.86      0.85      1442



In [ ]:
words_importance_nb = list([(voc[i], abs(class_nb.feature_log_prob_[0][i]))
                        for i in range(len(class_nb.feature_log_prob_[0]))])
sorted(words_importance_nb, key=lambda x: x[1], reverse=True)[:5]

[('00 бороться', 12.123484200214545),
 ('00 бороться либирахами', 12.123484200214545),
 ('00 бороться либирахами хохлами', 12.123484200214545),
 ('00 бороться либирахами хохлами проверил', 12.123484200214545),
 ('00 бороться либирахами хохлами проверил новости', 12.123484200214545)]

### **Случайный лес**

In [ ]:
# Random Forest
class_rf = RandomForestClassifier(max_depth=10000,
                                  criterion='log_loss')
class_rf.fit(X_train, y)
pred_rf = class_rf.predict(X_test)
print(classification_report(y_test, pred_rf, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.86      0.86       983
         1.0       0.70      0.72      0.71       459

    accuracy                           0.81      1442
   macro avg       0.78      0.79      0.79      1442
weighted avg       0.81      0.81      0.81      1442



In [ ]:
words_importance_rf = list([(voc[i], class_rf.feature_importances_[i])
                        for i in range(len(class_rf.feature_importances_))])
sorted(words_importance_rf, key=lambda x: x[1], reverse=True)[:5]

[('хохлы', 0.008161553258928048),
 ('хохлов', 0.007543611893195516),
 ('нахуи', 0.004726406128963404),
 ('например', 0.0036621815667386044),
 ('сука', 0.0035714319553756834)]